In [1]:
import cv2
import numpy as np

########################################################################################################################################
# function author: Murtaza 
# link: https://www.youtube.com/watch?v=Fchzk1lDt7Q&list=PLMoSUbG1Q_r_sc0x7ndCsqdIkL7dwrmNF&index=10
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

########################################################################################################################################
# function author: Murtaza 
# link: https://www.youtube.com/watch?v=Fchzk1lDt7Q&list=PLMoSUbG1Q_r_sc0x7ndCsqdIkL7dwrmNF&index=10
def getContours(img, imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    

    # We dont want contours that are below a certain area size
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1000:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)

            peri = cv2.arcLength(cnt, True) # find perimeters
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True) # find how many points there are
            # print(len(approx))

            # for bounding box
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x, y), (x + w, y + h), (0, 255, 0), 5)

########################################################################################################################################


In [ ]:
import cv2
import numpy as np

# frame to display webcam
frameWidth = 640
frameHeight = 4800

# capture video
webcam = cv2.VideoCapture(0)

"""
The ID 3 corresponds to the property for setting the width of the video frame.
The ID 4 corresponds to the property for setting the height of the video frame.
"""
webcam.set(3, frameWidth)
webcam.set(4, frameHeight)

def doNothing(a):
    """
    This function gets called by the trackbar when updated, we dont want to do anything
    """
    pass

# create a trackbar window menu
cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters", 640, 240)

# threshold values for canny edge detection
cv2.createTrackbar("Canny upper", "Parameters", 150, 255, doNothing)
cv2.createTrackbar("Canny lower", "Parameters", 150, 255, doNothing)

while True:
    success, img = webcam.read()
    
    # 1. apply gaussian blur - (7, 7) is kernel size
    imgBlurred = cv2.GaussianBlur(img, (7, 7), 1)

    # 2. turn image to greyscale
    imgGrayed = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)

    # 3. edge detection
    threshold1Val = cv2.getTrackbarPos("Canny upper", "Parameters")
    threshold2Val = cv2.getTrackbarPos("Canny lower", "Parameters")
    imgEdgeDetect = cv2.Canny(imgGrayed, threshold1Val, threshold2Val)

    # 4. dilate image
    kernel = np.ones((5, 5))
    imgDilated = cv2.dilate(imgEdgeDetect, kernel, iterations=1)

    # 5. contouring
    imgContour = img.copy()
    getContours(imgDilated, imgContour)

    # create an aggregate of images to display
    displayed = stackImages(0.8, ([img, imgEdgeDetect, imgDilated],
                            [imgContour, imgContour, imgContour]))


    # display the images
    cv2.imshow("Webcam stream", displayed)

    # press 'q' to close the webcam without crash
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break




In [3]:
# Importing OpenCV package
import cv2
import numpy as np

def doNothing(a):
    """
    This function gets called by the trackbar when updated, we dont want to do anything
    """
    pass

def detectObjects(img, img_contour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Applying object detction method
    objects = object_cascade.detectMultiScale(img_contours, 1.1, 9)

    # We dont want contours that are below a certain area size
    for obj in objects:
        area = cv2.contourArea(obj)
        if area > 1000:
            cv2.drawContours(imgContour, obj, -1, (255, 0, 255), 7)
        
        for (x, y, w, h) in objects:
            cv2.rectangle(img_contours,(x,y), (x+w,y+h), (255,0,0), 2)

# create a trackbar window menu
cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters", 640, 240)

# threshold values for canny edge detection
cv2.createTrackbar("Thresh 1", "Parameters", 150, 255, doNothing)
cv2.createTrackbar("Thresh 2", "Parameters", 150, 255, doNothing)

# Loading the required haar-cascade xml classifier file
object_cascade = cv2.CascadeClassifier('./cascades/tank.xml')

# Reading the image
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()

    
    img_blur = cv2.GaussianBlur(img, (7, 7), 1)
    img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)

    threshold1Val = cv2.getTrackbarPos("Thresh 1", "Parameters")
    threshold2Val = cv2.getTrackbarPos("Thresh 2", "Parameters")
    img_edge = cv2.Canny(img_blur, threshold1Val, threshold2Val)

    kernel = np.ones((5, 5))
    img_dilated = cv2.dilate(img_edge, kernel, iterations=1)

    img_contour = img.copy()
    getContours(img_dilated, img_contour)

    img_contours = img.copy()
    detectObjects(img_dilated, img_contours)

    # display the images
    cv2.imshow("Webcam stream", img_contours)

    # press 'q' to close the webcam without crash
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break

cap.release()
cv2.destroyAllWindows()
    
    


#
# Pseudocode for training a Haarcascade model

In [ ]:
'''
Collect positive images with the object you want to detect
Collect negative images without the object

Create a text file 'positive.txt' with paths to positive images and object coordinates
Create a text file 'negative.txt' with paths to negative images

Generate positive samples:
for each positive image in 'positive.txt':
    apply variations to the image (rotation, scaling, noise)
    save the modified image

Create a vector file 'positives.vec' using modified positive images
opencv_createsamples -info positive.txt -vec positives.vec -w width -h height

Train the cascade:
opencv_traincascade -data output_directory -vec positives.vec -bg negative.txt -numStages N

Evaluate the classifier:
for each test image:
    use the trained cascade to detect objects
    calculate precision, recall, and accuracy metrics

Tune parameters and retrain:
adjust cascade parameters (numStages, minHitRate, maxFalseAlarmRate)
retrain the cascade using the updated parameters

Use the trained cascade:
load the trained cascade using cv2.CascadeClassifier
use the cascade to detect objects in images or videos
'''
